# Part 2: 10-K Analyzer

## Part 2 uses exact URLs extracted from Part1 of the relevant 10-K as an input and performs the required tasks

### Note: Both Part1 and Part 2 can be easily integrated together. However, due to paucity of time I have currently not done it. I have manually taken the output of Part1 and used as an input for this part.

### <U> CHECKLIST (Refer the PDF file to understand the inevitability of manual intervention for correct output)

### 1. For question 1: 
### Ensure that the code is capturing correct starting point by checking hte outut of <u>'print(text10k[0:500])'</u> command and comaring it with actual 10-K. If required, change the index number of <u>'sec1TitleStarts'</u> VECTOR IN <u>'textBodyStart = sec1TitleStarts[0]'</u> command in 'Manual entry for body start section'

### 2. For question 2,3 & 4: 
### Ensure that the code is capturing the correct start and end point for MD&A section by checking the output of <u>'print(sec7Texttillend[0:1000])'</u>, <u>'print(sec7TextEndstillend[0:1000])'</u> and <u>'print(sec7Text[0:1000])'</u> command and comparing it with actual 10-K. If needed, change the index number of <u>'sec7TitleStarts'</u> & <u>'sec7TextEnds'</u> vector in <u>'sec7Text = text10k[sec7TitleStarts[INDEX NUMBER]:sec7TextEnds[INDEX NUMBER]]'</u> command.

In [1]:
# IMPORT REQUIRED LIBRARIES
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

In [2]:
# SAMPLE 10K PAGE
#url = "https://www.sec.gov/Archives/edgar/data/4962/000119312516469798/d131774d10k.htm" #American Express Co
#url = "https://www.sec.gov/Archives/edgar/data/320193/000162828016020309/a201610-k9242016.htm" #Apple Inc
#url = "https://www.sec.gov/Archives/edgar/data/732712/000119312516473367/d35513d10k.htm" #Verizon Communications Inc
#url = "https://www.sec.gov/Archives/edgar/data/12927/000001292716000099/a201512dec3110k.htm" #Boeing Co
#url = "https://www.sec.gov/Archives/edgar/data/18230/000001823016000410/cat_10-kx12312015.htm" #Caterpillar Inc
#url = "https://www.sec.gov/Archives/edgar/data/19617/000001961716000902/corp10k2015.htm" #JPMorgan Chase & Co
#url = "https://www.sec.gov/Archives/edgar/data/93410/000009341016000049/cvx-123115x10kdoc.htm" #Chevron Corp
#url = "https://www.sec.gov/Archives/edgar/data/21344/000002134416000050/a2015123110-k.htm" #Coca-Cola Co (The)
#url = "https://www.sec.gov/Archives/edgar/data/1001039/000100103916000516/fy2016_q4x10k.htm" #Walt Disney Co (The)
#url = "https://www.sec.gov/Archives/edgar/data/30554/000003055416000106/dd-12312015x10k.htm" #E. I. DU PONT DE NEMOURS AND COMPANY
#url = "https://www.sec.gov/Archives/edgar/data/34088/000003408816000065/xom10k2015.htm" #Exxon Mobil Corp
#url = "https://www.sec.gov/Archives/edgar/data/40545/000004054516000145/ge10k2015.htm" #General Electric Co
#url = "https://www.sec.gov/Archives/edgar/data/354950/000035495016000060/hd-1312016x10xk.htm" #Home Depot Inc. (The)
#url = "https://www.sec.gov/Archives/edgar/data/50863/000005086316000105/a10kdocument12262015q4.htm" #Intel Corp
#url = "https://www.sec.gov/Archives/edgar/data/51143/000104746916010329/a2226548z10-k.htm" #International Business Machines Corp
#url = "https://www.sec.gov/Archives/edgar/data/200406/000020040616000071/form10-k20160103.htm" #Johnson & Johnson
#url = "https://www.sec.gov/Archives/edgar/data/63908/000006390816000103/mcd-12312015x10k.htm" #McDonald's Corp
#url = "https://www.sec.gov/Archives/edgar/data/310158/000031015816000063/mrk1231201510k.htm" #Merck & Co Inc.
#url = "https://www.sec.gov/Archives/edgar/data/66740/000155837016003162/mmm-20151231x10k.htm" #3M Co
#url = "https://www.sec.gov/Archives/edgar/data/320187/000032018716000336/nke-5312016x10k.htm" #Nike Inc
#url = "https://www.sec.gov/Archives/edgar/data/78003/000007800316000077/pfe-12312015x10kshell.htm" #Pfizer Inc
#url = "https://www.sec.gov/Archives/edgar/data/80424/000008042416000212/fy151610-kreport.htm" #Procter & Gamble Co (The)
#url = "https://www.sec.gov/Archives/edgar/data/731766/000073176616000058/unh2015123110-k.htm" #Unitedhealth Group Inc
#url = "https://www.sec.gov/Archives/edgar/data/101829/000010182916000050/a2015-12x31form10xk.htm" #United Technologies Corp
#url = "https://www.sec.gov/Archives/edgar/data/104169/000010416916000079/wmtform10-kx1312016.htm" #Walmart Inc
#url = "https://www.sec.gov/Archives/edgar/data/1618921/000114036116083198/form10k.htm" #Walgreens Boots Alliance Inc
#url = "https://www.sec.gov/Archives/edgar/data/789019/000119312516662209/d187868d10k.htm" #Microsoft Corp
#url = "https://www.sec.gov/Archives/edgar/data/858877/000085887716000117/csco-2016730x10k.htm" #Cisco Systems Inc
#url = "https://www.sec.gov/Archives/edgar/data/86312/000104746916010107/a2227255z10-k.htm" #Travelers Companies Inc (The)
#url = "https://www.sec.gov/Archives/edgar/data/886982/000119312516470289/d107036d10k.htm" #Goldman Sachs Group Inc (The)
url = "https://www.sec.gov/Archives/edgar/data/1403161/000140316116000058/v093016.htm" #Visa Inc


# Q1. Total count of words

## Step1 Extracting text from 10K Page

In [3]:
# EXTRACTING 10K
data = []
response10K = requests.get(url)
soup10K = BeautifulSoup(response10K.content, "html.parser")
text10k = soup10K.get_text() 
# NOTE: get_text() module automatically ignores all tables, HTML tags etc. But will continue to include numbers.

In [4]:
# CLEANING 10K FOR READABILITY
text10k = re.sub("[\n][\n]+", "\n", text10k) # CLEANING UNECESSARY NEW LINES
text10k = re.sub("[\s][\s]+", " ", text10k) # CLEANING UNECESSARY SPACES
text10k = text10k.replace("\u00A0", " ")#.decode("utf-8") # CLEANING NON BREAKING SPACES
#len(text10k)

In [5]:
# INITIALISING TEXT SEARCH PARAMETERS
#sec1Pattern = "item[^0-9a-zA-Z]*1[^0-9a-zA-Z]*business"
sec1Pattern = "item[^0-9a-zA-Z]*1\.[^0-9a-zA-Z]*business"# for Walmart, The Travelers Comp

# For sec7 pattern, different options have been used (due to inconsistent foramtting) 
sec7pattern = "item[^0-9a-zA-Z]*7\."## used for AMEX, APPLE, VERIZON etc.
#sec7pattern = "item[^0-9a-zA-Z]*7\:" ## used for JP Morgan 
#sec7pattern = "item[^0-9a-zA-Z]*7\.[^0-9a-zA-Z]*management’s discussion and" ##Additional pattern
#print (sec7pattern,"\n")

# For sec7a pattern, different options have been used (due to inconsistent foramtting)
sec7apattern = "item[^0-9a-zA-Z]*7a\.[^0-9a-zA-Z]*quantitative and qualitative"
#sec7apattern = "item[^0-9a-zA-Z]*7a\:[^0-9a-zA-Z]*quantitative and qualitative" ## used for JP Morgan


In [6]:
# EXTRACTING "BODY" OF TEXT, NEGLECTING TABLE OF CONTENTS
sec1Start = re.finditer(sec1Pattern, text10k.lower())
sec1TitleStarts = [match.start(0) for match in sec1Start]
print (sec1TitleStarts,"\n")
textBodyStart = None
if len(sec1TitleStarts) == 1:
    textBodyStart = sec1TitleStarts[0]
elif len(sec1TitleStarts) == 2:
    textBodyStart = sec1TitleStarts[1]
else:
    print("There are multiple occurences of \"Item 1. Body\", find the \
          one corresponding to the start of the report body, and enter in below \
          commented code block","\n")
    
# MANUAL ENTRY FOR BODY START
textBodyStart = sec1TitleStarts[0] # ENTER INDEX INS SQ. BRACKETS. PYTHON INDICES START AT 0

# VERY IMPORTANT NOTE: TABLE OF CONTENTS SECTION OF 10K MAY NOT BE FORMATTED AS 'TABLE' IN ALL 10-KS. HENCE, MANUAL
# INTERVENTION IS NEEDED TO SEE IF THE CODE IS MARKING THE STARTING POINT CORRECTLY. 
# EVENTUALLY, RIGHT INDEX NUMBER SHOULD BE CHOSEN TO GET THE CORRECT COUNT.
text10k = text10k[textBodyStart: len(text10k)]
print(text10k[0:500],"\n")

[7597, 48578] 

ITEM 1. BusinessOVERVIEWVisa is a global payments technology company that connects consumers, merchants, financial institutions, businesses, strategic partners and government entities in more than 200 countries and territories to fast, secure and reliable electronic payments. We enable global commerce through the transfer of value and information among these participants. Our advanced transaction processing network facilitates authorization, clearing and settlement of payment transactions and en 



## Step2 Finding total number of words

In [7]:
completeWordList = re.findall(r"\b\w+\b", text10k)
print("The approximate number of words in the body of the text is: " + str(len(completeWordList)),"\n")

The approximate number of words in the body of the text is: 68560 



# Q2 Word count in MD&A section

## Step1 Extracting the text of MD&A

In [8]:
sec7Start = re.finditer(sec7pattern, text10k.lower())
#print(sec7Start,"\n")
sec7TitleStarts = [match.start(0) for match in sec7Start]
sec7End = re.finditer(sec7apattern, text10k.lower())
sec7TextEnds = [match.start(0) for match in sec7End]

print(sec7TitleStarts, sec7TextEnds,"\n")
print(sec7TitleStarts,"\n")
print(sec7TextEnds,"\n")

# To capture the correct start and end point, manual intervention may be needed. HTML codes of all 10-Ks are not consistent, hence this is required. 
sec7Texttillend = text10k[sec7TitleStarts[0]: len(text10k)]
print(sec7Texttillend[0:1000],"\n")

sec7TextEndstillend = text10k[sec7TextEnds[0]: len(text10k)]
print(sec7TextEndstillend[0:1000],"\n")

sec7Text = text10k[sec7TitleStarts[0]:sec7TextEnds[0]]
print(sec7Text[0:1000],"\n")

[101284] [174885] 

[101284] 

[174885] 

ITEM 7.Management’s Discussion and Analysis of Financial Condition and Results of OperationsThis management’s discussion and analysis provides a review of the results of operations, financial condition and liquidity and capital resources of Visa Inc. and its subsidiaries (“Visa,” “we,” "us," “our” and the “Company”) on a historical basis and outlines the factors that have affected recent earnings, as well as those factors that may affect future earnings. The following discussion and analysis should be read in conjunction with the consolidated financial statements and related notes included in Item 8 of this report.OverviewVisa is a global payments technology company that connects consumers, merchants, financial institutions, businesses, strategic partners and government entities in more than 200 countries and territories to fast, secure and reliable electronic payments. We enable global commerce through the transfer of value and information amo

## Step2 Finding number of words in MD&A section

In [9]:
sec7WordList = re.findall(r"\b\w+\b", sec7Text)
print("Number of words in the MD&A section is: " + str(len(sec7WordList)),"\n")

Number of words in the MD&A section is: 11634 



# Q3 Count of word 'competition' in MD&A section

In [10]:
competitionList = re.findall("competition", sec7Text)
print("Number of times \"competition\" is mentioned is:" + str(len(competitionList)),"\n")

Number of times "competition" is mentioned is:1 



In [11]:
# INITIALISING REGEX DATE PATTERNS:
datePattern = r"[a-z,A-Z]+ [0-9][0-9], [0-9][0-9][0-9][0-9]"

## Q4 Finding dates and 10 words on either side

In [12]:
# FINDING THE DATES
sec7Dates = re.findall(datePattern, sec7Text.lower())
print(sec7Dates,"\n")

['june 21, 2016', 'june 21, 2016', 'september 30, 2016', 'june 21, 2016', 'june 30, 2016', 'september 30, 2016', 'june 14, 2016', 'september 30, 2014', 'june 21, 2016', 'september 30, 2016', 'june 30, 2016', 'september 30, 2016', 'june 30, 2016', 'june 30, 2016', 'june 30, 2016', 'june 21, 2016', 'june 30, 2016', 'june 21, 2016', 'june 21, 2016', 'september 30, 2016', 'june 21, 2016', 'june 30, 2016', 'september 30, 2016', 'june 21, 2016', 'june 21, 2016', 'june 30, 2016', 'june 21, 2016', 'june 30, 2016', 'september 30, 2016', 'june 21, 2016', 'june 30, 2016', 'september 30, 2016', 'september 30, 2016', 'september 30, 2016', 'january 27, 2016', 'january 27, 2021', 'january 27, 2016', 'september 30, 2016', 'december 14, 2017', 'june 14, 2016', 'september 30, 2016', 'september 30, 2016', 'june 21, 2016', 'june 21, 2016', 'september 30, 2016', 'september 30, 2016', 'september 30, 2016', 'september 30, 2016', 'november 18, 2016', 'june 21, 2016', 'september 30, 2016', 'september 30, 2016'

In [13]:
# FINDING 10 WORDS ON EITHER SIDE OF THE DATES
TenleftSideWords=[]
TenrightSideWords=[]
for date in set(sec7Dates):
    #TenleftSideWords=[]
    #TenrightSideWords=[]    
    dateSplits = re.split(date, sec7Text.lower())    
    numSplits = len(dateSplits)
    print (numSplits)
    print(len(dateSplits), "\n")
    for i in range(numSplits - 1):
        print(i, i+1, "\n")
        
        leftSideString = dateSplits[i]
        rightSideString = dateSplits[i+1]
        
        leftSideWords = leftSideString.split(" ")
        rightSideWords = rightSideString.split(" ")
        
        print(date,"\n")
        print(leftSideWords[-10:],"\n")
        print(rightSideWords[0:10],"\n")
        print("Moving to the next occurence of the date","\n")
        TenleftSideWords.append(leftSideWords[-10:])
        TenrightSideWords.append(rightSideWords[0:10])
        print(TenleftSideWords)
        print(TenrightSideWords)
    print("Moving to the next date","\n")
#10leftSideWords.append(10leftSideWords)
#10leftSideWords.append(10leftSideWords)
    # Insert relevant figures into the list of lists, to be converted to DataFrame
data.append([url, len(completeWordList),len(sec7WordList),len(competitionList), TenleftSideWords, TenrightSideWords])
    
    
    # Create a Pandas dataframe
df=pd.DataFrame(data, columns=['URL','Total Words','MD&A Words','Competition','LeftWords', 'RightWords'])

    #Create a Pandas Excel writer using XlsWriter as the engine'
writer=pd.ExcelWriter('pandas_simple.xlsx', engine ='xlsxwriter')

# Convert the dataframe to an X;sWriter Excel object.
df.to_excel(writer, sheet_name='Sheet1')

#Close the Pandas Excel writer and output the Excel file.
writer.save()

3
3 

0 1 

june 14, 2016 

['payable', 'semi-annually', 'on', 'june', '14', 'and', 'december', '14,', 'commencing', ''] 

['.', 'the', 'net', 'aggregate', 'proceeds', 'of', '$15.9', 'billion,', 'after', 'deducting'] 

Moving to the next occurence of the date 

[['payable', 'semi-annually', 'on', 'june', '14', 'and', 'december', '14,', 'commencing', '']]
[['.', 'the', 'net', 'aggregate', 'proceeds', 'of', '$15.9', 'billion,', 'after', 'deducting']]
1 2 

june 14, 2016 

['an', 'interest', 'payment', 'of', '$244', 'million', 'was', 'made', 'on', ''] 

['.', 'the', 'notes', 'may', 'be', 'redeemed', 'as', 'a', 'whole', 'or'] 

Moving to the next occurence of the date 

[['payable', 'semi-annually', 'on', 'june', '14', 'and', 'december', '14,', 'commencing', ''], ['an', 'interest', 'payment', 'of', '$244', 'million', 'was', 'made', 'on', '']]
[['.', 'the', 'net', 'aggregate', 'proceeds', 'of', '$15.9', 'billion,', 'after', 'deducting'], ['.', 'the', 'notes', 'may', 'be', 'redeemed', 'as', 

june 21, 2016 

['with', 'the', 'framework', 'agreement', 'prior', 'to', 'its', 'acquisition', 'on', ''] 

['.', 'fiscal', 'year', 'endedseptember', '30,', '$', 'change', '%', 'change(1)', '2016(2)'] 

Moving to the next occurence of the date 

[['payable', 'semi-annually', 'on', 'june', '14', 'and', 'december', '14,', 'commencing', ''], ['an', 'interest', 'payment', 'of', '$244', 'million', 'was', 'made', 'on', ''], ['revolving', 'credit', 'facility.', 'the', 'credit', 'facility,', 'which', 'expires', 'on', ''], ['continued', 'uneven', 'and', 'tepid', 'economic', 'growth.visa', 'europe', 'acquisition.', 'on', ''], ['billion)', 'at', 'the', 'closing', 'stock', 'price', 'of', '$77.33', 'on', ''], ['our', 'consolidated', 'statements', 'of', 'operations', 'from', 'the', 'acquisition', 'date,', ''], ['framework', 'agreement', 'loss.', 'upon', 'consummation', 'of', 'the', 'transaction,', 'on', ''], ['processed', 'transactions', 'of', 'visa', 'europe', 'from', 'the', 'acquisition', 'date,', 

['.', 'we', 'did', 'not', 'include', 'visa', "europe's", 'financial', 'results', 'in'] 

Moving to the next occurence of the date 

[['payable', 'semi-annually', 'on', 'june', '14', 'and', 'december', '14,', 'commencing', ''], ['an', 'interest', 'payment', 'of', '$244', 'million', 'was', 'made', 'on', ''], ['revolving', 'credit', 'facility.', 'the', 'credit', 'facility,', 'which', 'expires', 'on', ''], ['continued', 'uneven', 'and', 'tepid', 'economic', 'growth.visa', 'europe', 'acquisition.', 'on', ''], ['billion)', 'at', 'the', 'closing', 'stock', 'price', 'of', '$77.33', 'on', ''], ['our', 'consolidated', 'statements', 'of', 'operations', 'from', 'the', 'acquisition', 'date,', ''], ['framework', 'agreement', 'loss.', 'upon', 'consummation', 'of', 'the', 'transaction,', 'on', ''], ['processed', 'transactions', 'of', 'visa', 'europe', 'from', 'the', 'acquisition', 'date,', ''], ['incurred', 'upon', 'consummation', 'of', 'the', 'visa', 'europe', 'acquisition', 'on', ''], ['framework', 